In [29]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from keras import layers
import keras
from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')

In [30]:
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
genres = 'cry no_cry'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'b_cry/{g}'):
        songname = f'b_cry/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 576x576 with 0 Axes>

In [31]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [32]:
file = open('dataset.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'cry no_cry'.split()
for g in genres:
    for filename in os.listdir(f'b_cry/{g}'):
        songname = f'b_cry/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=7)
        rmse = librosa.feature.rms(y=y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('dataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [33]:
data = pd.read_csv('dataset.csv')
data.head()# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)#Encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))#Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [34]:
model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [35]:
model.fit(X_train,y_train,shuffle=True,epochs=20,batch_size=128)

Epoch 1/20
2/2 [==============================] - ETA: 0s - loss: 2.0876 - accuracy: 0.31 - 0s 5ms/step - loss: 2.0832 - accuracy: 0.3111
Epoch 2/20
2/2 [==============================] - ETA: 0s - loss: 1.6729 - accuracy: 0.70 - 0s 7ms/step - loss: 1.6626 - accuracy: 0.7037
Epoch 3/20
2/2 [==============================] - ETA: 0s - loss: 1.3382 - accuracy: 0.82 - 0s 8ms/step - loss: 1.3247 - accuracy: 0.8370
Epoch 4/20
2/2 [==============================] - ETA: 0s - loss: 1.0281 - accuracy: 0.87 - 0s 7ms/step - loss: 1.0284 - accuracy: 0.8741
Epoch 5/20
2/2 [==============================] - ETA: 0s - loss: 0.7852 - accuracy: 0.89 - 0s 5ms/step - loss: 0.7827 - accuracy: 0.8815
Epoch 6/20
2/2 [==============================] - ETA: 0s - loss: 0.6013 - accuracy: 0.87 - 0s 5ms/step - loss: 0.5939 - accuracy: 0.8815
Epoch 7/20
2/2 [==============================] - ETA: 0s - loss: 0.4719 - accuracy: 0.87 - 0s 5ms/step - loss: 0.4611 - accuracy: 0.8815
Epoch 8/20
2/2 [==================

In [36]:
model.evaluate_generator(generator=X_test, steps=50)#OUTPUT

[0.0, 0.0]

In [1]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()


* recording
* done recording


In [2]:
# cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
pathlib.Path(f'live_data/').mkdir(parents=True, exist_ok=True)
# for filename in os.listdir(f'b_cry/{g}'):
songname = f'output.wav'
y, sr = librosa.load(songname, mono=True, duration=7)
plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
plt.axis('off');
plt.savefig(f'live_data/output.png')
plt.clf()

NameError: name 'plt' is not defined

In [3]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [4]:
file = open('predict.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
# genres = 'discomfort belly_pain burping hungry'.split()
# for g in genres:
#     for filename in os.listdir(f'b_cry/{g}'):
# songname = f'b_cry/{g}/{filename}'
songname = f'output.wav'
y, sr = librosa.load(songname, mono=True, duration=5)
rmse = librosa.feature.rms(y=y)
chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr)
to_append = f'output.wav {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'   
for e in mfcc:
    to_append += f' {np.mean(e)}'
# to_append += f' {model.predict(live_data.iloc[0])}'
#to_append += f' live_data_genre'
file = open('predict.csv', 'a', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(to_append.split())
live_data = pd.read_csv('predict.csv')
live_data = live_data.drop(['filename'],axis=1)


NameError: name 'csv' is not defined

In [5]:
#print(live_data.iloc[0][0:26])
Xnew = np.array([live_data.iloc[0][0:26]])
print(Xnew)
ynew = model.predict_classes(Xnew)
print("Predicted=%s" % (ynew))
if ynew==0:
    print('Cry')
else:
    print('No cry')
#live_data = data.drop(['filename'],axis=1)
#print(model.predict(live_data.iloc[:, -1]))
# print(type(live_data.iloc[0]))
#print(model.predict_classes(live_data.iloc[0]))
# print(live_data.iloc[0])

NameError: name 'np' is not defined

In [57]:
#p_pred=model.predict_proba(X_test)

In [58]:
score = model.evaluate(X_train, y_train, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))
score = model.evaluate(X_test, y_test, verbose=0)
print("Testing Accuracy: {0:.2%}".format(score[1]))

Training Accuracy: 98.52%
Testing Accuracy: 79.41%


In [59]:
y_pred=model.predict_classes(X_test)
y_pred

array([0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1], dtype=int64)

In [60]:
y_test=y_test.astype('int64')
y_test

array([0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0], dtype=int64)

In [61]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred)

0.7941176470588235